<a href="https://colab.research.google.com/github/MarcoFierimonte/virtual-try-off/blob/main/Colab_Script_for_Qwen_Clothing_Try_On.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run the **Qwen Image Edit Clothing Try-On** application on a free Google Colab GPU (T4, 16GB VRAM), we must use memory optimizations because the base model is quite large (~20B parameters).

The following script installs the latest `diffusers` library, loads the **Qwen-Image-Edit-2509** model with 4-bit quantization (to fit into the T4's VRAM), and implements the two-step pipeline (Extract $\rightarrow$ Try-On) described in the DigitalOcean tutorial.

### Google Colab Script

In [ ]:
# 1. Install necessary dependencies
!pip install -U git+https://github.com/huggingface/diffusers.git
!pip install -U transformers accelerate peft bitsandbytes sentencepiece gradio

import torch
from PIL import Image
from diffusers import QwenImageEditPlusPipeline
from diffusers.utils import load_image
import gradio as gr

# 2. Setup the Pipeline with Memory Optimizations
# We use 4-bit quantization and CPU offloading to fit a 20B model on a T4 GPU
model_id = "Qwen/Qwen-Image-Edit-2509"

print("Loading model... This may take a few minutes.")
pipe = QwenImageEditPlusPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto", # Automatically handles memory across CPU/GPU
    load_in_4bit=True  # Crucial for Colab Free T4
)

# 3. Define the Extraction Function
def extract_clothes(input_image):
    # Load the specific LoRA for clothing extraction
    pipe.load_lora_weights(
        "JamesDigitalOcean/Qwen_Image_Edit_Extract_Clothing",
        weight_names="qwen_image_edit_remove_body.safetensors",
        adapter_name="removebody"
    )

    # Process image
    if isinstance(input_image, str):
        pil_image = load_image(input_image)
    else:
        pil_image = Image.fromarray(input_image).convert("RGB")

    prompt = "removebody remove the person from this image, but leave the outfit. the clothes should remain after deleting the person's body, skin, and hair. leave the clothes in front of a white background"

    image = pipe(
        image=[pil_image],
        prompt=prompt,
        num_inference_steps=30 # Reduced for speed
    ).images[0]

    pipe.unload_lora_weights() # Clear weights to save VRAM
    return image

# 4. Define the Try-On Function
def tryon_clothes(clothing_image, person_image):
    # Load the specific LoRA for virtual try-on
    pipe.load_lora_weights(
        "JamesDigitalOcean/Qwen_Image_Edit_Try_On_Clothes",
        weight_names="qwen_image_edit_tryon.safetensors",
        adapter_name="tryonclothes"
    )

    # Ensure images are PIL
    pil_clothes = Image.fromarray(clothing_image).convert("RGB") if not isinstance(clothing_image, Image.Image) else clothing_image
    pil_person = Image.fromarray(person_image).convert("RGB") if not isinstance(person_image, Image.Image) else person_image

    # Prompt logic from tutorial: Person on top, clothing on bottom
    prompt = "tryonclothes put the set of clothing onto the person in the image. Style the person in the image using the clothing provided."

    image = pipe(
        image=[pil_person, pil_clothes],
        prompt=prompt,
        num_inference_steps=30
    ).images[0]

    pipe.unload_lora_weights()
    return image

# 5. Create Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("# Qwen Image Edit: Clothing Try-On")

    with gr.Row():
        with gr.Column():
            input_model = gr.Image(label="1. Original Photo (Model wearing clothes)")
            extract_btn = gr.Button("Step 1: Extract Clothing")
        with gr.Column():
            extracted_output = gr.Image(label="Extracted Clothing (White Background)")

    with gr.Row():
        with gr.Column():
            target_person = gr.Image(label="2. Target Person (Photo to receive clothes)")
            tryon_btn = gr.Button("Step 2: Apply Try-On")
        with gr.Column():
            final_output = gr.Image(label="Final Result")

    # Wire up the buttons
    extract_btn.click(fn=extract_clothes, inputs=input_model, outputs=extracted_output)
    tryon_btn.click(fn=tryon_clothes, inputs=[extracted_output, target_person], outputs=final_output)

demo.launch(debug=True, share=True)

### Key Setup Instructions for Colab:

1. **Change Runtime**: Go to `Runtime` -> `Change runtime type` and select **T4 GPU**.
2. **Memory Warning**: Even with 4-bit quantization, this model is extremely large for a T4. If you encounter an "Out of Memory" (OOM) error, try restarting the session and running the code again without running other cells.
3. **Authentication**: If the model asks for a Hugging Face token, you may need to run `!huggingface-cli login` in a cell beforehand, as some Qwen weights require accepting a license agreement on their Hugging Face page.

[Qwen Image Edit Try-On Tutorial](https://www.youtube.com/watch?v=CY9U59mcWqQ)

This video provides a visual walkthrough of the Qwen Image Edit LoRA workflow for virtual try-ons, which is helpful for understanding how to align the input images and prompts for the best results.